In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
# 导入数据
dataraw=pd.read_csv(r"E:\桌面\统计建模数据\data21.csv",encoding='utf-8')
data=pd.DataFrame(dataraw,columns=['lninv', 'lninv_i','lninv_ui','csr', 'soe', 'size', 'age', 'dual', 'rd', 'leve', 'roe', 'indd', 'top1', 'jigou', 'meiti','csrsq'])
# 进行特征转换，对数变换
# data_transformed = data.copy()
# data_transformed[['csr', 'soe', 'size', 'age', 'dual', 'rd', 'leve', 'roe', 'indd', 'top1', 'jigou', 'meiti']] = np.log1p(data[['csr', 'soe', 'size', 'age', 'dual', 'rd', 'leve', 'roe', 'indd', 'top1', 'jigou', 'meiti']])
# 分割数据集为训练集和测试集
X = data.drop(['lninv','lninv_i','lninv_ui','csrsq'], axis=1)  # 自变量
y = data['lninv']  # 因变量

mse_test=[]
r2_test=[]
oob_r2=[]
#0，001测试集的r2最好0.9286060532724022，但是测试样本只有两个，0.007测试样本的r2较低，但是测试样本有15个0.8217251660858522
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.007, random_state=389)

# 创建带外数据的随机森林模型(如果用data21的数据，就是114和389的参数)
# rf_oob = RandomForestRegressor(n_estimators=756, oob_score=True, random_state=409,max_depth=None,min_samples_leaf=1,min_samples_split=2)
rf_oob = RandomForestRegressor(n_estimators=114, oob_score=True, random_state=389,max_depth=26,min_samples_leaf=1,min_samples_split=2)
#在训练集上拟合模型，同时使用带外数据进行评估
rf_oob.fit(X_train, y_train)

    # 使用测试集进行预测
y_pred_test = rf_oob.predict(X_test)
y_pred_train=rf_oob.predict(X_train)

    # 计算均方误差和决定系数
mse_train=mean_squared_error(y_train,y_pred_train)
r2_train=r2_score(y_train,y_pred_train)
mse_test.append(mean_squared_error(y_test, y_pred_test))
r2_test.append(r2_score(y_test, y_pred_test))
oob_predictions = rf_oob.oob_prediction_
oob_mse_train = mean_squared_error(y_train, oob_predictions)
    # 访问带外数据上的决定系数
oob_r2.append(rf_oob.oob_score_)


print(f"训练集的均方误差: {mse_train}")
print(f"测试集的均方误差: {mse_test}")
print(f"训练集上R²: {r2_train}")
print(f"测试集上R²: {r2_test}")
print(f"OOB训练集的均方误差: {oob_mse_train}")
print(f"OOB测试集上的R²: {oob_r2}")